In [31]:
import polars as pl
import json
import numpy as np
pl.Config.set_fmt_float("mixed")

polars.config.Config

In [3]:
with open('data/raw/data_2020-01-01.json', 'r') as f:
    data = json.load(f)

In [ ]:
def format_data(data):
    data_list = []
    for date in data['near_earth_objects']:
        result = {'date': date}
        for idx, obj in enumerate(data['near_earth_objects'][date]):
            new_result = result | {
                # General stuff about astroid
                'obj_that_day': idx,
                'id': str(obj['id']),
                'name': obj['name'],
                'absolute_magniutude_h': obj['absolute_magnitude_h'],
                # Estimated Diameter
                'estimated_diameter_min_km': obj['estimated_diameter']['kilometers']['estimated_diameter_min'],
                'estimated_diameter_max_km': obj['estimated_diameter']['kilometers']['estimated_diameter_max'],
                'estimated_diameter_min_m': obj['estimated_diameter']['meters']['estimated_diameter_min'],
                'estimated_diameter_max_m': obj['estimated_diameter']['meters']['estimated_diameter_max'],
                'estimated_diameter_min_miles': obj['estimated_diameter']['miles']['estimated_diameter_min'],
                'estimated_diameter_max_miles': obj['estimated_diameter']['miles']['estimated_diameter_max'],
                'estimated_diameter_min_feet': obj['estimated_diameter']['feet']['estimated_diameter_min'],
                'estimated_diameter_max_feet': obj['estimated_diameter']['feet']['estimated_diameter_max'],
                # Potentially hazardous, Sentry object refers to if the astroid is tracked by nasa's sentry system 
                'is_potentially_hazardous': obj['is_potentially_hazardous_asteroid'],
                'is_sentry_object': obj['is_sentry_object']
            }
            for val in obj['close_approach_data']:
                final_result = new_result | {
                    # Close approaching dates
                    'close_approach_date': val['close_approach_date_full'],
                    'epoch_date_close_approach': val['epoch_date_close_approach'],
                    # Velocity values
                    'relative_velocity_km/sec': float(val['relative_velocity']['kilometers_per_second']),
                    'relative_velocity_km/hr': float(val['relative_velocity']['kilometers_per_hour']),
                    'relative_velocity_mph': float(val['relative_velocity']['miles_per_hour']),
                    # Miss distance
                    'miss_distance_astronomical': float(val['miss_distance']['astronomical']),
                    'miss_distance_lunar': float(val['miss_distance']['lunar']),
                    'miss_distance_kilometers': float(val['miss_distance']['kilometers']),
                    'miss_distance_miles': float(val['miss_distance']['miles']),
                    # Orbiting body 
                    'oribiting_body': val['orbiting_body']
                }

                data_list.append(final_result)
    return data_list

In [38]:
df = pl.DataFrame(data_list)

In [39]:
df.columns

['date',
 'obj_that_day',
 'id',
 'name',
 'absolute_magniutude_h',
 'estimated_diameter_min_km',
 'estimated_diameter_max_km',
 'estimated_diameter_min_m',
 'estimated_diameter_max_m',
 'estimated_diameter_min_miles',
 'estimated_diameter_max_miles',
 'estimated_diameter_min_feet',
 'estimated_diameter_max_feet',
 'is_potentially_hazardous',
 'is_sentry_object',
 'close_approach_date',
 'epoch_date_close_approach',
 'relative_velocity_km/sec',
 'relative_velocity_km/hr',
 'relative_velocity_mph',
 'miss_distance_astronomical',
 'miss_distance_lunar',
 'miss_distance_kilometers',
 'miss_distance_miles',
 'oribiting_body']

In [62]:
# Feature engineering
def feature_engineering(data: pl.DataFrame) -> pl.DataFrame:
    df = data.with_columns([
        # Size based features
        ((pl.col('estimated_diameter_min_km') + pl.col('estimated_diameter_max_km')) / 2).alias('avg_diameter_km'),
        (pl.col('estimated_diameter_max_km') - pl.col('estimated_diameter_min_km')).alias('diameter_uncertainty_km'),
        
        # Volume of sphere (4/3 x pi x r^3)
        ((4/3) * np.pi * (((pl.col('estimated_diameter_min_km') + pl.col('estimated_diameter_max_km')) / 4) ** 3)).alias('estimated_volume'),
        # Cross section area from earth (pi x r^2)
        (np.pi * (((pl.col('estimated_diameter_min_km') + pl.col('estimated_diameter_max_km')) / 4) ** 2)).alias('cross_section_area_km2')
    
    ]).with_columns([
        # Diameter uncertentity ratio 
        (pl.col('diameter_uncertainty_km') / pl.col('avg_diameter_km')).alias('diameter_uncertainty_ratio'),
        # Site category
        pl.when(pl.col('avg_diameter_km') < 0.05).then(pl.lit('tiny'))
        .when(pl.col('avg_diameter_km') < 0.14).then(pl.lit('small'))
        .when(pl.col('avg_diameter_km') < 0.5).then(pl.lit('medium'))
        .otherwise(pl.lit('large')).alias('size_category'),

        # Velocity & Kenetic energy features
        # Kenetic Energy (KE = 1/2 * m * v^2)
        (pl.col('estimated_volume') * pl.col('relative_velocity_km/sec') ** 2).alias('kenetic_energy'),
        # Momentum (volume x velocity)
        (pl.col('estimated_volume') * pl.col('relative_velocity_km/sec')).alias('momentum'),
        # velocity per atronomic unit
        (pl.col('relative_velocity_km/sec') / pl.col('miss_distance_astronomical')).alias('velocity_per_au'),
        # velocity distance ratio (fast + close = dangerous, slow + far = safe)
        (pl.col('relative_velocity_km/sec') / (pl.col('miss_distance_kilometers')/ 1e6)).alias('velocity_distance_ratio'),
        # Velocity category
        pl.when(pl.col('relative_velocity_km/sec') < 10).then(pl.lit('slow'))
        .when(pl.col('relative_velocity_km/sec') < 20).then(pl.lit('medium'))
        .when(pl.col('relative_velocity_km/sec') < 30).then(pl.lit('fast'))
        .otherwise(pl.lit('very_fast')).alias('velocity_category'),

    ]).with_columns([
        # Distance & Risk factors
        # Lunar distance ratio
        (pl.col('miss_distance_lunar') / 384400).alias("lunar_distance_ratio"),
        # Earth radii distance (earth in radius unites)
        (pl.col('miss_distance_kilometers')/ 6371).alias('earth_radii_distance'),
        # Close approach score (closer = higher score)
        (1 / pl.col('miss_distance_astronomical')).alias('close_approach_score'),
        # Impact potential (combined size, speed and proximity threat)
        ((pl.col('avg_diameter_km') * pl.col('relative_velocity_km/sec')) / pl.col('miss_distance_astronomical')).alias('impact_potential'),
        # Destruction potential (energy density at earths distance)
        (pl.col('kenetic_energy') / pl.col('miss_distance_astronomical')).alias('destruction_potential'),
        # Hazard index (combined size^2, velocity^2 and distance): (diameter^2 x velocity^2) / distance_km
        ((pl.col('avg_diameter_km') ** 2 * pl.col('relative_velocity_km/sec') ** 2) / pl.col('miss_distance_kilometers')).alias('hazard_index'),
        # Proximity level
        pl.when(pl.col('miss_distance_lunar') < 10).then(pl.lit('very_close'))
        .when(pl.col('miss_distance_lunar') < 50).then(pl.lit('close'))
        .when(pl.col('miss_distance_lunar') < 100).then(pl.lit('moderate'))
        .otherwise(pl.lit('far')).alias('proximity_level'),
        
        # Approach datetime - Convert epoch timestamp to readable datetime
        pl.from_epoch('epoch_date_close_approach', time_unit='ms').alias('approach_datetime')

    ]).with_columns([
        # Temporal features
        # Time components
        pl.col('approach_datetime').dt.year().alias('approach_year'),
        pl.col('approach_datetime').dt.month().alias('approach_month'),
        pl.col('approach_datetime').dt.day().alias('approach_day'),
        pl.col('approach_datetime').dt.hour().alias('approach_hour'),

        # Day of week and year
        pl.col('approach_datetime').dt.weekday().alias('day_of_week'),
        pl.col('approach_datetime').dt.ordinal_day().alias('day_of_year')
   
    ]).with_columns([
        # Month_sin/month_cos + hour_sin/hour_cos (cyclical encoding of moinths)
        (2 * np.pi * pl.col('approach_month') / 12).sin().alias('month_sin'),
        (2 * np.pi * pl.col('approach_month') / 12).cos().alias('month_cos'),
        (2 * np.pi * pl.col('approach_hour') / 24).sin().alias('hour_sin'),
        (2 * np.pi * pl.col('approach_hour') / 24).cos().alias('hour_cos'),

        # Brightness & physical features
        # Brightness size ratio
        (pl.col('absolute_magniutude_h') / pl.col('avg_diameter_km')).alias('brightness_size_ratio'),
        # Apparant densitiy inversed (1/volume) (higher = denser)
        # Dense (metal) vs Fluffy (rubble pile)
        (1 / pl.col('estimated_volume')).alias('apparent_density_inverse'),

        # Brightness category
        pl.when(pl.col('absolute_magniutude_h') < 20).then(pl.lit('very_bright'))
        .when(pl.col('absolute_magniutude_h') < 22).then(pl.lit('bright'))
        .when(pl.col('absolute_magniutude_h') < 25).then(pl.lit('dim'))
        .otherwise(pl.lit('very_dim')).alias('brightness_category'),

        # Interaction features

        # Size velocity produt (size and speed)
        (pl.col('avg_diameter_km') * pl.col('relative_velocity_km/sec')).alias("size_velocity_product"),
        # Size squared velocity (emphasises on size) (diameter^2 x velocity)
        (pl.col('avg_diameter_km') ** 2 * pl.col('relative_velocity_km/sec')).alias("size_squared_velocity"),
        # Escape velocity ratio (speed compared to easrth escape velocity)
        (pl.col('relative_velocity_km/sec') / 11.2).alias('escape_velocity_ratio'),
        # Threat score (size x velocity) / (distance + weighed_value_to_avoid_div_0)
        ((pl.col('avg_diameter_km') * pl.col('relative_velocity_km/sec')) / (pl.col('miss_distance_astronomical') + 0.001)).alias('threat_score'),

        # Normalisation & Scaling features
        # Size percentile (0-1)
        (pl.col('avg_diameter_km').rank(method='average') / pl.col('avg_diameter_km').len()).alias('size_percentile'),
        # Velocity percentile
        (pl.col('relative_velocity_km/sec').rank(method='average') / pl.col('relative_velocity_km/sec').len()).alias('velocity_percentile'),
        # Distance percentile
        (pl.col('miss_distance_astronomical').rank(method='average') / pl.col('miss_distance_astronomical').len()).alias('distance_percentile')
    ]).with_columns([
        # Z-scores & Log values
        # Size zscore
        ((pl.col('avg_diameter_km') - pl.col('avg_diameter_km').mean()) / pl.col('avg_diameter_km').std()).alias('size_zscore'),
        # Velocity zscore
        ((pl.col('relative_velocity_km/sec') - pl.col('relative_velocity_km/sec').mean()) / pl.col('relative_velocity_km/sec').std()).alias('velocity_zscore'),
        # distance zscore
        ((pl.col('miss_distance_astronomical') - pl.col('miss_distance_astronomical').mean()) / pl.col('miss_distance_astronomical').std()).alias('distance_zscore'),

        # Log diameter 
        pl.col('avg_diameter_km').log1p().alias('log_diameter'),
        # log velocity
        pl.col('relative_velocity_km/sec').log1p().alias('log_velocity'),
        # log distance
        pl.col('miss_distance_kilometers').log1p().alias('log_distance')
    ])

    return df

In [63]:
new_data = feature_engineering(data=df)

In [65]:
new_data.write_csv('feature_engineered_data.csv')

In [69]:
print("\n=== MISSING VALUES ===")
null_counts = new_data.null_count()
print("Columns with missing values:")
null_counts


=== MISSING VALUES ===
Columns with missing values:


date,obj_that_day,id,name,absolute_magniutude_h,estimated_diameter_min_km,estimated_diameter_max_km,estimated_diameter_min_m,estimated_diameter_max_m,estimated_diameter_min_miles,estimated_diameter_max_miles,estimated_diameter_min_feet,estimated_diameter_max_feet,is_potentially_hazardous,is_sentry_object,close_approach_date,epoch_date_close_approach,relative_velocity_km/sec,relative_velocity_km/hr,relative_velocity_mph,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,miss_distance_miles,oribiting_body,avg_diameter_km,diameter_uncertainty_km,estimated_volume,cross_section_area_km2,diameter_uncertainty_ratio,size_category,kenetic_energy,momentum,velocity_per_au,velocity_distance_ratio,velocity_category,lunar_distance_ratio,earth_radii_distance,close_approach_score,impact_potential,destruction_potential,hazard_index,proximity_level,approach_datetime,approach_year,approach_month,approach_day,approach_hour,day_of_week,day_of_year,month_sin,month_cos,hour_sin,hour_cos,brightness_size_ratio,apparent_density_inverse,brightness_category,size_velocity_product,size_squared_velocity,escape_velocity_ratio,threat_score,size_percentile,velocity_percentile,distance_percentile,size_zscore,velocity_zscore,distance_zscore,log_diameter,log_velocity,log_distance
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
print("=== DATA SHAPE & TYPES ===")
print(f"Shape: {new_data.shape}")
print(f"Columns: {len(new_data.columns)}")
print("\nData types:")
print(new_data.dtypes)

print("\n=== TARGET VARIABLE ===")
print(f"Target distribution:")
print(new_data['is_potentially_hazardous'].value_counts())

print("\n=== MISSING VALUES ===")
null_counts = new_data.null_count().to_series().to_list()
print("Columns with missing values:")
print(null_counts)

print("\n=== SAMPLE ROWS ===")
print(new_data.head(3))

=== DATA SHAPE & TYPES ===
Shape: (126, 70)
Columns: 70

Data types:
[String, Int64, String, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Boolean, Boolean, String, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, Float64, Float64, String, Float64, Float64, Float64, Float64, Float64, Float64, String, Datetime(time_unit='ms', time_zone=None), Int32, Int8, Int8, Int8, Int8, Int16, Float64, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64]

=== TARGET VARIABLE ===
Target distribution:
shape: (2, 2)
┌──────────────────────────┬───────┐
│ is_potentially_hazardous ┆ count │
│ ---                      ┆ ---   │
│ bool                     ┆ u32   │
╞══════════════════════════╪═══════╡
│ true                     ┆ 12    │
│ false       

In [77]:
numerical_cols = new_data.select(pl.col(pl.Float64, pl.Int64, pl.Int32, pl.Int8, pl.Int16)).columns
numerical_cols = [col for col in numerical_cols if col not in ['obj_that_day', 'epoch_date_close_approach']]

In [ ]:
def feature_selection_corr(data):
    

    correlations = []
    for col in numerical_cols:
        try:
            corr = data.select(pl.corr('is_potentially_hazardous', col)).item()
            if corr is not None:
                correlations.append((col, abs(col)))
        except:
            continue
    
    correlations.sort(key=lambda x: x[1], reverse=True)
    return correlations


In [72]:
correlations = feature_selection_corr(new_data) 

In [93]:
import matplotlib.pyplot as plt
import seaborn as sns

temp_data = new_data.select(numerical_cols)

correlations = {}
for col in numerical_cols:
    corr = new_data.select(pl.corr('is_potentially_hazardous', col)).item()
    correlations[col] = corr

In [111]:
selected_features = [
    # Strongest predictors (>0.3 correlation)
    'threat_score',           # 0.351 - Best overall predictor!
    'impact_potential',       # 0.345 - Physics-based threat
    'size_percentile',        # 0.403 - Relative size ranking
    'log_diameter',           # 0.301 - Transformed size measure
    
    # Strong predictors (0.2-0.3 correlation)  
    'avg_diameter_km',        # 0.244 - Core size measure
    'velocity_zscore',        # 0.203 - Normalized velocity
    'relative_velocity_km/sec', # 0.203 - Core velocity
    'log_velocity',           # 0.226 - Transformed velocity
    'velocity_percentile',    # 0.235 - Relative velocity ranking
    
    # Supporting features (0.15-0.2)
    'size_velocity_product',  # 0.153 - Size × speed interaction
    
    # Key categorical/expert features
    'is_sentry_object',       # Expert NASA assessment (boolean)
    
    # Negative correlation (important!)
    'absolute_magniutude_h',  # -0.377 - Brightness (smaller=brighter=larger!)
    'brightness_size_ratio'   # -0.212 - Physical relationship
    ]
def train_test_split(data):
    X = data.select(selected_features)
    y = data.select('is_potentially_hazardous')

    X_pd = X.to_pandas()
    y_pd = y.to_pandas().squeeze()

    from sklearn.model_selection import train_test_split, StratifiedKFold

    X_train, X_test, y_train, y_test = train_test_split(
        X_pd, y_pd,
        test_size=0.25,
        stratify=y_pd,
        random_state=42
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


    return X_train, X_test, y_train, y_test, cv

In [112]:
X_train, X_test, y_train, y_test, cv = train_test_split(data=new_data)

In [113]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
def smart_preprocessing(X_train, X_test):
    boolean_features = ['is_sentry_object']
    numerical_features = [col for col in selected_features if col not in boolean_features]

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('bool', 'passthrough', boolean_features)
        ],
        remainder='drop'
    )

    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.fit_transform(X_test)

    return X_train_processed, X_test_processed

In [114]:
X_train_processed, X_test_processed = smart_preprocessing(X_train, X_test)

In [136]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score

def baseline_model(X_train_processed, X_test_processed, y_train, y_test, cv):
    # Model 1: Random Forest
    rf = RandomForestClassifier(
        n_estimators=200, # More trees for better performance
        class_weight='balanced', # Handle 12 vs 114 imbalance
        random_state=42,
        max_depth=4, # Shallow to prevent overfitting
        min_samples_split=5, # Conservastive splitting
        min_samples_leaf=2 # Ensure meaning full leaves
    )

    # Cross val scores
    rf_cv_scores = cross_val_score(rf, X_train_processed, y_train, cv=cv, scoring='f1')
    print(f"F1 Scores: {rf_cv_scores}")
    print(f"Mean F1: {rf_cv_scores.mean():.3f} (+/- {rf_cv_scores.std() * 2:.3f})")

    # Fit and evaluate
    rf.fit(X_train_processed, y_train)
    rf_pred = rf.predict(X_test_processed)
    rf_prob = rf.predict_proba(X_test_processed)[:, 1]
    print("-------------RANDOM FOREST TEST RESULTS-------------")
    print(classification_report(y_test, rf_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, rf_prob):.3f}")

    # Model 2: Logistic Regression
    lr = LogisticRegression(
        class_weight='balanced',
        random_state=42,
        max_iter=2000,
        C=0.1
    )

    lr_cv_scores = cross_val_score(lr, X_train_processed, y_train, cv=cv, scoring='f1')
    print(f"F1 Scores: {lr_cv_scores}")
    print(f"Mean F1: {lr_cv_scores.mean():.3f} (+/- {lr_cv_scores.std() * 2:.3f})")

    lr.fit(X_train_processed, y_train)
    lr_pred = lr.predict(X_test_processed)
    lr_prob = lr.predict_proba(X_test_processed)[:, 1]
    print("-------------LOGISITIC REGRESSION TEST RESULTS-------------")
    print(classification_report(y_test, lr_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, lr_prob):.3f}")

    return rf_pred, rf_prob, lr_pred, lr_prob, rf, lr

In [137]:
rf_pred, rf_prob, lr_pred, lr_prob, rf, lr = baseline_model(X_train_processed=X_train_processed, X_test_processed=X_test_processed, y_train=y_train, y_test=y_test, cv=cv)

F1 Scores: [0.66666667 0.5        0.66666667 0.         0.5       ]
Mean F1: 0.467 (+/- 0.490)
-------------RANDOM FOREST TEST RESULTS-------------
              precision    recall  f1-score   support

       False       0.93      0.97      0.95        29
        True       0.50      0.33      0.40         3

    accuracy                           0.91        32
   macro avg       0.72      0.65      0.67        32
weighted avg       0.89      0.91      0.90        32

ROC-AUC: 0.908
F1 Scores: [0.5        0.66666667 0.66666667 0.66666667 0.4       ]
Mean F1: 0.580 (+/- 0.222)
-------------LOGISITIC REGRESSION TEST RESULTS-------------
              precision    recall  f1-score   support

       False       0.96      0.83      0.89        29
        True       0.29      0.67      0.40         3

    accuracy                           0.81        32
   macro avg       0.62      0.75      0.64        32
weighted avg       0.90      0.81      0.84        32

ROC-AUC: 0.885


In [ ]:
import pandas as pd
def feature_importance(model):
    feature_names = [col for col in selected_features]
    importances = model.feature_importances_

    feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
    }).sort_values('importance', ascending=False)

    return feature_importance_df

In [139]:
feature_importance(rf)

,feature,importance
11,absolute_magniutude_h,1.451242e-01
10,is_sentry_object,1.335752e-01
9,size_velocity_product,1.331765e-01
0,threat_score,1.248301e-01
2,size_percentile,1.022533e-01
4,avg_diameter_km,8.831750e-02
1,impact_potential,8.515958e-02
3,log_diameter,7.042571e-02
7,log_velocity,3.584730e-02
6,relative_velocity_km/sec,2.963773e-02


In [140]:
%pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
    --------------------------------------- 2.1/150.0 MB 11.0 MB/s eta 0:00:14
   - -------------------------------------- 4.2/150.0 MB 10.8 MB/s eta 0:00:14
   - -------------------------------------- 6.6/150.0 MB 10.8 MB/s eta 0:00:14
   -- ------------------------------------- 8.9/150.0 MB 10.7 MB/s eta 0:00:14
   -- ------------------------------------- 11.0/150.0 MB 10.8 MB/s eta 0:00:13
   --- ------------------------------------ 13.1/150.0 MB 10.8 MB/s eta 0:00:13
   ---- ----------------------------------- 15.5/150.0 MB 10.9 MB/s eta 0:00:13
   ---- ----------------------------------- 17.8/150.0 MB 10.8 MB/s eta 0:00:13
   ----- ---------------------------------- 19.7/150.0 MB 10.6 MB/s eta 0:00:13
   ----- ---------------------------------- 21.8/150.0 MB 10.6 MB/s eta 0:00:13
   ------ --------------------------------- 24.1/150.0 MB 10.6 MB/s eta 0:00:12
   ------- -------------------------------- 26.5/150.

In [152]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb

def xgb_model():
    # XGBBoost
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        scale_pos_weight=len(y_train[y_train == False]) / len(y_train[y_train == False]),
        random_state=42
    )

    xgb_cv_scores = cross_val_score(xgb_model, X_train_processed, y_train, cv=cv, scoring='f1')
    print("🚀 XGBOOST CROSS-VALIDATION:")
    print(f"Mean F1: {xgb_cv_scores.mean():.3f} (+/- {xgb_cv_scores.std() * 2:.3f})")

    # Test performance
    xgb_model.fit(X_train_processed, y_train)
    xgb_pred = xgb_model.predict(X_test_processed)
    xgb_prob = xgb_model.predict_proba(X_test_processed)[:, 1]

    print("\n🚀 XGBOOST TEST RESULTS:")
    print(classification_report(y_test, xgb_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, xgb_prob):.3f}")

    return xgb_pred, xgb_prob, xgb_model

In [153]:
xgb_pred, xgb_prob, xgb_model = xgb_model()

🚀 XGBOOST CROSS-VALIDATION:
Mean F1: 0.133 (+/- 0.533)

🚀 XGBOOST TEST RESULTS:
              precision    recall  f1-score   support

       False       0.93      0.97      0.95        29
        True       0.50      0.33      0.40         3

    accuracy                           0.91        32
   macro avg       0.72      0.65      0.67        32
weighted avg       0.89      0.91      0.90        32

ROC-AUC: 0.931


In [161]:
def svc_model():
    svm = SVC(
        probability=True, 
        class_weight='balanced',
        kernel='rbf',
        C=1.0,
        random_state=42
    )

    svm_cross_scores = cross_val_score(svm, X_train_processed, y_train, cv=cv, scoring='f1')
    print("🚀 SVM CROSS-VALIDATION:")
    print(f"Mean F1: {svm_cross_scores.mean():.3f} (+/- {svm_cross_scores.std() * 2:.3f})")

    # Test performance
    svm.fit(X_train_processed, y_train)
    svm_pred = svm.predict(X_test_processed)
    svm_prob = svm.predict_proba(X_test_processed)[:, 1]

    print("\n🚀 SVM TEST RESULTS:")
    print(classification_report(y_test, svm_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, svm_prob):.3f}")

    return svm_pred, svm_prob, svm

In [162]:
svm_pred, svm_prob, svm = svc_model()

🚀 SVM CROSS-VALIDATION:
Mean F1: 0.482 (+/- 0.199)

🚀 SVM TEST RESULTS:
              precision    recall  f1-score   support

       False       0.96      0.90      0.93        29
        True       0.40      0.67      0.50         3

    accuracy                           0.88        32
   macro avg       0.68      0.78      0.71        32
weighted avg       0.91      0.88      0.89        32

ROC-AUC: 0.908


In [163]:
def gradient_boosting_model():
    gb = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    )

    gb_cross_scores = cross_val_score(gb, X_train_processed, y_train, cv=cv, scoring='f1')
    print("🚀 Gradient Boosting CROSS-VALIDATION:")
    print(f"Mean F1: {gb_cross_scores.mean():.3f} (+/- {gb_cross_scores.std() * 2:.3f})")

    # Test performance
    gb.fit(X_train_processed, y_train)
    gb_pred = gb.predict(X_test_processed)
    gb_prob = gb.predict_proba(X_test_processed)[:, 1]

    print("\n🚀 Gradient Boosting TEST RESULTS:")
    print(classification_report(y_test, gb_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, gb_prob):.3f}")

    return gb_pred, gb_prob, gb

    

In [164]:
gb_pred, gb_prob, gb = gradient_boosting_model()

🚀 Gradient Boosting CROSS-VALIDATION:
Mean F1: 0.447 (+/- 0.491)

🚀 Gradient Boosting TEST RESULTS:
              precision    recall  f1-score   support

       False       0.94      1.00      0.97        29
        True       1.00      0.33      0.50         3

    accuracy                           0.94        32
   macro avg       0.97      0.67      0.73        32
weighted avg       0.94      0.94      0.92        32

ROC-AUC: 0.908


In [165]:
from sklearn.neural_network import MLPClassifier

def basic_neural_net():
    mlp = MLPClassifier(
        hidden_layer_sizes=(20,10),
        max_iter=1000,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.2
    )

    mlp_cross_scores = cross_val_score(mlp, X_train_processed, y_train, cv=cv, scoring='f1')
    print("🚀 Basic Neural Net CROSS-VALIDATION:")
    print(f"Mean F1: {mlp_cross_scores.mean():.3f} (+/- {mlp_cross_scores.std() * 2:.3f})")

    # Test performance
    mlp.fit(X_train_processed, y_train)
    mlp_pred = mlp.predict(X_test_processed)
    mlp_prob = mlp.predict_proba(X_test_processed)[:, 1]

    print("\n🚀 Basic Neural Net TEST RESULTS:")
    print(classification_report(y_test, mlp_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, mlp_prob):.3f}")

    return mlp_pred, mlp_prob, mlp

In [166]:
mlp_pred, mlp_prob, mlp = basic_neural_net()

🚀 Basic Neural Net CROSS-VALIDATION:
Mean F1: 0.363 (+/- 0.325)

🚀 Basic Neural Net TEST RESULTS:
              precision    recall  f1-score   support

       False       1.00      0.59      0.74        29
        True       0.20      1.00      0.33         3

    accuracy                           0.62        32
   macro avg       0.60      0.79      0.54        32
weighted avg       0.93      0.62      0.70        32

ROC-AUC: 0.954


In [168]:
from sklearn.model_selection import GridSearchCV
def tune_svv():
    svm_param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.1, 1],
        'kernel': ['rbf', 'poly'],
        'class_weight': ['blanced', {False: 1, True: 5}, {False: 1, True: 10}]
    }

    svm_grid = GridSearchCV(
        svm,
        svm_param_grid,
        cv=cv,
        scoring='f1',
        n_jobs=-1
    )

    print(f"Tuning SVM")
    svm_grid.fit(X_train_processed, y_train)

    print("🎯 BEST SVM PARAMETERS:")
    print(svm_grid.best_params_)
    print(f"Best CV F1: {svm_grid.best_score_:.3f}")

    best_svm = svm_grid.best_estimator_
    best_svm_pred = best_svm.predict(X_test_processed)
    best_svm_prob = best_svm.predict_proba(X_test_processed)[:, 1]

    print('OPTIMISED SVM RESULTS')
    print(classification_report(y_test, best_svm_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, best_svm_prob):.3f}")

    return best_svm_pred, best_svm_prob, best_svm

In [169]:
svm_pred_new, svm_prob_new, svm_new_model = tune_svv()

Tuning SVM
🎯 BEST SVM PARAMETERS:
{'C': 10, 'class_weight': {False: 1, True: 5}, 'gamma': 0.01, 'kernel': 'rbf'}
Best CV F1: 0.581
OPTIMISED SVM RESULTS
              precision    recall  f1-score   support

       False       0.96      0.90      0.93        29
        True       0.40      0.67      0.50         3

    accuracy                           0.88        32
   macro avg       0.68      0.78      0.71        32
weighted avg       0.91      0.88      0.89        32

ROC-AUC: 0.897


c:\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python313\Lib\site-packages\sklearn\base.py", line 1356, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Python313\Lib\site-packages\sklearn\base.py", line 469, in _validate_params
    validate_parameter_constraints(
    ~~~~~~~~~

In [170]:
from sklearn.model_selection import RandomizedSearchCV
def svm_tuning_advanced():
    svm_params = {
        'C': [0.01, 0.1, 1, 10, 50, 100, 200],
        'gamma':  ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
        'kernel': ['rbf', 'poly', 'sigmoid'],
        'class_weight': [
            'balanced',
            {False: 1, True: 3},
            {False: 1, True: 5},
            {False: 1, True: 8},
            {False: 1, True: 10},
            {False: 1, True: 15},
            {False: 1, True: 20},
        ],
        'degree': [2,3,4]
    }

    svm_random_search = RandomizedSearchCV(
        svm_new_model,
        svm_params,
        n_iter=200,
        cv=cv,
        scoring='f1',
        n_jobs=1,
        random_state=42,
        verbose=1
    )

    print(f"Starting SVM random search")
    svm_random_search.fit(X_train_processed, y_train)
    print("\n🏆 BEST SVM PARAMETERS:")
    print(svm_random_search.best_params_)
    print(f"Best CV F1: {svm_random_search.best_score_:.3f}")

    # Test best SVM
    best_svm_tuned = svm_random_search.best_estimator_
    svm_tuned_pred = best_svm_tuned.predict(X_test_processed)
    svm_tuned_prob = best_svm_tuned.predict_proba(X_test_processed)[:, 1]

    print("\n🎯 OPTIMIZED SVM TEST RESULTS:")
    print(classification_report(y_test, svm_tuned_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, svm_tuned_prob):.3f}")

    return svm_tuned_pred, svm_tuned_prob, best_svm_tuned

In [171]:
svm_tuned_pred, svm_tuned_prob, best_svm_tuned = svm_tuning_advanced()

Starting SVM random search
Fitting 5 folds for each of 200 candidates, totalling 1000 fits

🏆 BEST SVM PARAMETERS:
{'kernel': 'rbf', 'gamma': 0.01, 'degree': 4, 'class_weight': {False: 1, True: 15}, 'C': 10}
Best CV F1: 0.589

🎯 OPTIMIZED SVM TEST RESULTS:
              precision    recall  f1-score   support

       False       0.96      0.90      0.93        29
        True       0.40      0.67      0.50         3

    accuracy                           0.88        32
   macro avg       0.68      0.78      0.71        32
weighted avg       0.91      0.88      0.89        32

ROC-AUC: 0.920


In [172]:
def random_forest_tuning():
    rf_params = {
        'n_estimators': [100, 200, 300, 500, 800],
        'max_depth': [3, 4, 5,6,7,8, None],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8],
        'max_features': ['sqrt', 'log2', 0.3, 0.5, 0.7],
        'class_weight':[
            'balanced',
            'balanced_subsample',
            {False: 1, True: 3},
            {False: 1, True: 5},
            {False: 1, True: 8},
            {False: 1, True: 10},
            {False: 1, True: 16}
        ],
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy']
    }

    rf_random_search = RandomizedSearchCV(
        rf,
        rf_params,
        n_iter=300,
        cv=cv,
        scoring='f1',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    print("Starting Random Forest random search (this may take 15-20 minutes)...")
    rf_random_search.fit(X_train_processed, y_train)

    print("\n🏆 BEST RANDOM FOREST PARAMETERS:")
    print(rf_random_search.best_params_)
    print(f"Best CV F1: {rf_random_search.best_score_:.3f}")

    # Test best Random Forest
    best_rf_tuned = rf_random_search.best_estimator_
    rf_tuned_pred = best_rf_tuned.predict(X_test_processed)
    rf_tuned_prob = best_rf_tuned.predict_proba(X_test_processed)[:, 1]

    print("\n🎯 OPTIMIZED RANDOM FOREST TEST RESULTS:")
    print(classification_report(y_test, rf_tuned_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, rf_tuned_prob):.3f}")

    return rf_tuned_pred, rf_tuned_prob, best_rf_tuned


In [173]:
rf_tuned_pred, rf_tuned_prob, best_rf_tuned = random_forest_tuning()

Starting Random Forest random search (this may take 15-20 minutes)...
Fitting 5 folds for each of 300 candidates, totalling 1500 fits

🏆 BEST RANDOM FOREST PARAMETERS:
{'n_estimators': 100, 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 7, 'criterion': 'entropy', 'class_weight': 'balanced_subsample', 'bootstrap': True}
Best CV F1: 0.640

🎯 OPTIMIZED RANDOM FOREST TEST RESULTS:
              precision    recall  f1-score   support

       False       0.93      0.90      0.91        29
        True       0.25      0.33      0.29         3

    accuracy                           0.84        32
   macro avg       0.59      0.61      0.60        32
weighted avg       0.86      0.84      0.85        32

ROC-AUC: 0.897


In [176]:
from sklearn.inspection import permutation_importance

def feature_importance_analysis(model):
    perm_importance = permutation_importance(
        model, X_test_processed, y_test,
        n_repeats=20, random_state=42, scoring='f1'
    )

    feature_importance_df = pd.DataFrame({
        'feature': selected_features,
        'importance': perm_importance.importances_mean,
        'std': perm_importance.importances_std
    }).sort_values('importance', ascending=False)

    return feature_importance_df

In [177]:
feature_importance_df = feature_importance_analysis(best_svm_tuned)

In [178]:
feature_importance_df

,feature,importance,std
0,threat_score,0.079563,0.103559
1,impact_potential,0.047421,0.088985
7,log_velocity,0.039286,0.128940
3,log_diameter,0.024603,0.049621
2,size_percentile,0.022738,0.250494
11,absolute_magniutude_h,0.014286,0.089214
8,velocity_percentile,0.009167,0.151441
4,avg_diameter_km,0.005556,0.016667
6,relative_velocity_km/sec,0.001984,0.023660
5,velocity_zscore,0.001984,0.023660
